In [4]:
# Packages
import sqlite3
import random
import pandas as pd
import requests
import json
import time
import math

Randomized Sample 5

In [5]:
def draw_fifth_random_sample():
    source_db = "sampling_frame.db"
    sample1_db = "random_sample_1.db"
    sample2_db = "random_sample_2.db"
    sample3_db = "random_sample_3.db"
    sample4_db = "random_sample_4.db"
    fifth_sample_db = "random_sample_5.db"
    table_name = "sampled_collections"
    sample_size = 40

    # Load full dataset
    conn_source = sqlite3.connect(source_db)
    df_full = pd.read_sql_query(f"SELECT * FROM {table_name}", conn_source)
    conn_source.close()

    # Load previous samples
    conn1 = sqlite3.connect(sample1_db)
    df1 = pd.read_sql_query(f"SELECT * FROM {table_name}", conn1)
    conn1.close()

    conn2 = sqlite3.connect(sample2_db)
    df2 = pd.read_sql_query(f"SELECT * FROM {table_name}", conn2)
    conn2.close()

    conn3 = sqlite3.connect(sample3_db)
    df3 = pd.read_sql_query(f"SELECT * FROM {table_name}", conn3)
    conn3.close()

    conn4 = sqlite3.connect(sample4_db)
    df4 = pd.read_sql_query(f"SELECT * FROM {table_name}", conn4)
    conn4.close()    

    # Check for required column
    if "slug_name" not in df_full.columns:
        raise ValueError("Missing 'slug_name' column in source data.")

    # Exclude all previously sampled collection_ids
    previously_sampled_ids = set(df1["slug_name"]).union(df2["slug_name"], df3["slug_name"], df4["slug_name"])
    df_remaining = df_full[~df_full["slug_name"].isin(previously_sampled_ids)]

    print(f"Remaining collections after excluding first 4 samples: {len(df_remaining)}")

    # Draw the fourth sample
    df_fifth_sample = df_remaining.sample(n=sample_size, random_state=789)
    print(f"Sampled {len(df_fifth_sample)} collections for fifth sample.")

    # Write to new database
    conn_fifth = sqlite3.connect(fifth_sample_db)
    df_fifth_sample.to_sql(table_name, conn_fifth, if_exists="replace", index=False)
    conn_fifth.close()

    print(f"Fifth sample saved to '{df_fifth_sample}' in table '{table_name}'.")

In [6]:
draw_fifth_random_sample()

Remaining collections after excluding first 4 samples: 87
Sampled 40 collections for fifth sample.
Fifth sample saved to '          id                                      collection_id marketplace  \
31    448440   0x716ad1b6222046289c1664825cd9e4caf6253aec:match     Rarible   
39   5195586   0xdc9c44cefa0a48bd94f10a922a7b54af8a989e95:blast     OpenSea   
62   5423467                                               None   MagicEden   
77   4669448    0xb87d431ce921c33b7fd69bf7aa665caac63d6d31:zora     OpenSea   
17   4329255  0xa534852c55647201ae5413dfd398b5b35a1b6dc4:ape...     OpenSea   
179  5537670                                               None      Atomic   
57    143242    0x0c15b61197c8fca4322b8e6c4a744b597622a1dc:base     Rarible   
161  3756970    0x84f2789475754572311d65173ade3c24e643ba29:base     Rarible   
33   4383665    0xffc20825c2d56c8eb4fee84bfd1be00465b84195:zora     OpenSea   
48   3503331    0x97683a99e531201de43f68c445a0325a426ebf7c:celo     Rarible   
69   5432

In [7]:
def drop_other_tables(db_path="random_sample_5.db", keep_table="sampled_collections"):
    """
    Connects to 'db_path', enumerates all tables, 
    and drops any table that is not 'keep_table' or 'sqlite_sequence'.
    """
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # 1) Get all tables in the SQLite file
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
    tables = [row[0] for row in cursor.fetchall()]

    # 2) For each table, if it's not keep_table or 'sqlite_sequence', drop it
    for tbl in tables:
        if tbl not in [keep_table, "sqlite_sequence"]:
            drop_sql = f"DROP TABLE IF EXISTS '{tbl}'"
            cursor.execute(drop_sql)
            print(f"Dropped table: {tbl}")

    conn.commit()
    conn.close()
    print(f"All tables except '{keep_table}' and 'sqlite_sequence' have been dropped from '{db_path}'.")


if __name__ == "__main__":
    # Example usage
    drop_other_tables(db_path="random_sample_5.db", keep_table="sampled_collections")

Dropped table: opensea_nfts
Dropped table: rarible_nfts
Dropped table: magiceden_nfts
Dropped table: atomic_nfts
All tables except 'sampled_collections' and 'sqlite_sequence' have been dropped from 'random_sample_5.db'.


# Additional Data

In [8]:
def create_opensea_nfts_table(db_path="random_sample_5.db", table_name="opensea_nfts"):
    """
    Creates a table with columns for the relevant fields from the OpenSea NFT JSON.
    """
    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    
    # Create table if not exists
    c.execute(f"""
    CREATE TABLE IF NOT EXISTS {table_name} (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        
        -- Linking field from 'sampled_collections' table
        slug_name TEXT,
        
        -- Fields from the NFT JSON:
        identifier TEXT,
        contract TEXT,
        token_standard TEXT,
        name TEXT,
        description TEXT,
        image_url TEXT,
        display_image_url TEXT,
        display_animation_url TEXT,
        metadata_url TEXT,
        opensea_url TEXT,
        updated_at TEXT,
        is_disabled INTEGER,
        is_nsfw INTEGER,
        
        fetched_at TEXT
    )
    """)
    
    conn.commit()
    conn.close()

In [9]:
def fetch_opensea_nfts(collection_slug, api_key, limit=10):
    """
    Calls the OpenSea endpoint:
      GET /api/v2/collection/{collection_slug}/nfts?limit={limit}
    Returns the parsed JSON dict with:
       { "nfts": [ { ... }, {...} ], "next": "string" }
    If there's an error, returns an empty dict.
    """
    base_url = "https://api.opensea.io/api/v2/collection"
    url = f"{base_url}/{collection_slug}/nfts"
    
    headers = {
        "accept": "application/json",
        "x-api-key": api_key
    }
    params = {
        "limit": limit
    }
    
    resp = requests.get(url, headers=headers, params=params)
    if resp.status_code == 200:
        return resp.json()
    else:
        print(f"[OpenSea NFT Fetch] Error {resp.status_code} for '{collection_slug}': {resp.text}")
        return {}


def store_opensea_nfts(db_path, table_name, slug_name, nfts_data):
    """
    Inserts each NFT from nfts_data["nfts"] into opensea_nfts, linking them
    via 'slug_name'. We do NOT store the 'collection' field from the JSON,
    but instead rely on 'slug_name' to identify the parent collection.
    """
    nfts_list = nfts_data.get("nfts", [])
    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    
    insert_sql = f"""
    INSERT INTO {table_name} (
        slug_name,
        identifier,
        contract,
        token_standard,
        name,
        description,
        image_url,
        display_image_url,
        display_animation_url,
        metadata_url,
        opensea_url,
        updated_at,
        is_disabled,
        is_nsfw,
        fetched_at
    )
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, datetime('now'))
    """
    
    for nft in nfts_list:
        identifier = nft.get("identifier", "")
        contract = nft.get("contract", "")
        token_standard = nft.get("token_standard", "")
        name = nft.get("name", "")
        desc = nft.get("description", "")
        image_url = nft.get("image_url", "")
        display_image = nft.get("display_image_url", "")
        display_animation = nft.get("display_animation_url", "")
        metadata_url = nft.get("metadata_url", "")
        opensea_url = nft.get("opensea_url", "")
        updated_at = nft.get("updated_at", "")
        
        # Convert booleans to int (1/0)
        is_disabled = 1 if nft.get("is_disabled", False) else 0
        is_nsfw = 1 if nft.get("is_nsfw", False) else 0
        
        c.execute(insert_sql, (
            slug_name,
            identifier,
            contract,
            token_standard,
            name,
            desc,
            image_url,
            display_image,
            display_animation,
            metadata_url,
            opensea_url,
            updated_at,
            is_disabled,
            is_nsfw
        ))
    
    conn.commit()
    conn.close()


def fetch_opensea_nfts_for_collections(db_path="random_sample_5.db", table_collections="sampled_collections",
                                       table_nfts="opensea_nfts", api_key="YOUR_API_KEY", limit=10):
    """
    1) Ensure 'opensea_nfts' table exists.
    2) Query 'sampled_collections' for marketplace='OpenSea', using 'slug_name'
    3) For each, fetch up to {limit} items from OpenSea.
    4) Store them in 'opensea_nfts'.
    """
    # Create table if not exists
    create_opensea_nfts_table(db_path, table_nfts)
    
    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    
    # Query the relevant rows
    c.execute(f"""
    SELECT slug_name
    FROM {table_collections}
    WHERE marketplace='OpenSea'
      AND slug_name IS NOT NULL
    """)
    rows = c.fetchall()
    conn.close()
    
    for idx, (slug,) in enumerate(rows, start=1):
        print(f"[{idx}/{len(rows)}] Fetching up to {limit} NFTs for slug='{slug}'")
        nfts_data = fetch_opensea_nfts(slug, api_key, limit=limit)
        store_opensea_nfts(db_path, table_nfts, slug, nfts_data)
    
    print("Done fetching NFT data for all OpenSea collections in random_sample_5.db.")

In [ ]:
if __name__ == "__main__":
    OPENSEA_API_KEY = "your_key"
    fetch_opensea_nfts_for_collections(
        db_path="random_sample_5.db",
        table_collections="sampled_collections",
        table_nfts="opensea_nfts",
        api_key=OPENSEA_API_KEY,
        limit=10
    )

[1/14] Fetching up to 10 NFTs for slug='mekaapes-blast'
[2/14] Fetching up to 10 NFTs for slug='cincinnati-physicians'
[3/14] Fetching up to 10 NFTs for slug='dump-pepe-5'
[4/14] Fetching up to 10 NFTs for slug='zora-posts-21385'
[5/14] Fetching up to 10 NFTs for slug='me-physical-collectibles-10'
[6/14] Fetching up to 10 NFTs for slug='aaaaaa-26'
[7/14] Fetching up to 10 NFTs for slug='giants-g-nine-2024-emblem'
[8/14] Fetching up to 10 NFTs for slug='zora-posts-19156'
[9/14] Fetching up to 10 NFTs for slug='bbbdf-e'
[10/14] Fetching up to 10 NFTs for slug='mintak-2'
[11/14] Fetching up to 10 NFTs for slug='lens-collect-profile-126134-publication-890'
[12/14] Fetching up to 10 NFTs for slug='city-s-art'
[13/14] Fetching up to 10 NFTs for slug='blueandpink'
[14/14] Fetching up to 10 NFTs for slug='lens-collect-profile-143257-publication-8280'
Done fetching NFT data for all OpenSea collections in random_sample_5.db.


In [11]:
def add_new_columns_if_not_exists(db_path="random_sample_5.db", table_name="sampled_collections"):
    """
    Adds the column 'total_supply' to the 'sampled_collections' table
    """
    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    
    cols_to_add = [
        ("total_supply", "REAL")
    ]
    
    for col, col_type in cols_to_add:
        alter_stmt = f"ALTER TABLE {table_name} ADD COLUMN {col} {col_type}"
        try:
            c.execute(alter_stmt)
            print(f"Added column '{col}' to {table_name}")
        except sqlite3.OperationalError as e:
            # Likely column already exists
            pass
    
    conn.commit()
    conn.close()

In [12]:
def fetch_opensea_collection_details(slug, api_key):
    """
    Calls the endpoint:
      GET https://api.opensea.io/api/v2/collections/{collection_slug}
    Returns a dict with keys:
       'collection', 'name', 'total_supply', 'created_date', 'fees' (list), etc.
    """
    url = f"https://api.opensea.io/api/v2/collections/{slug}"
    headers = {
        "accept": "application/json",
        "x-api-key": api_key
    }
    resp = requests.get(url, headers=headers)
    if resp.status_code == 200:
        return resp.json() 
    else:
        print(f"[OpenSea Collection Fetch] Error {resp.status_code} for slug='{slug}': {resp.text}")
        return {}

def update_opensea_collection_data(db_path, table_name, slug, data):
    """
    data has: 'created_date', 'total_supply', 'fees' list, etc.
    We map:
       created_date -> created_time
       total_supply -> total_supply
       fees[0].fee -> marketplace_fee
       fees[1].fee -> royalty_fee (if exist)
    Then do UPDATE on the row with marketplace='OpenSea' AND slug_name=?
    """
    collection_slug = data.get("collection", slug)  # fallback to slug if missing
    created_date = data.get("created_date", None)   # string
    total_supply = data.get("total_supply", 0.0)
    fees = data.get("fees", [])
    
    marketplace_fee = None
    royalty_fee = None
    
    if len(fees) >= 1:
        # first fee => marketplace fee
        marketplace_fee = fees[0].get("fee", 0.0)
    if len(fees) >= 2:
        # second fee => royalty fee
        royalty_fee = fees[1].get("fee", 0.0)
    
    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    
    update_stmt = f"""
    UPDATE {table_name}
    SET created_time = ?,
        total_supply = ?,
        marketplace_fee = ?,
        royalty_fee = ?
    WHERE marketplace='OpenSea'
      AND slug_name=?
    """
    c.execute(update_stmt, (
        created_date,
        total_supply,
        marketplace_fee,
        royalty_fee,
        slug
    ))
    conn.commit()
    conn.close()

def fetch_and_update_opensea_collection_data(db_path="random_sample_5.db", table_name="sampled_collections",
                                             api_key="YOUR_API_KEY"):
    """
    # 1) Ensure the new columns exist in the 'sampled_collections' table.
    # 2) Query all rows for marketplace='OpenSea'.
    # 3) For each slug_name, call the new endpoint to fetch additional data,
    #    then update the row with created_time, total_supply, marketplace_fee, royalty_fee.
    # """

    add_new_columns_if_not_exists()
    
    # 2) Query the existing table
    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    c.execute(f"""
        SELECT slug_name
        FROM {table_name}
        WHERE marketplace='OpenSea'
          AND slug_name IS NOT NULL
    """)
    rows = c.fetchall()
    conn.close()
    
    for idx, (slug,) in enumerate(rows, start=1):
        print(f"[{idx}/{len(rows)}] Fetching extended data for slug='{slug}'")
        data = fetch_opensea_collection_details(slug, api_key)
        if data:
            # parse JSON: top-level fields => data
            # The endpoint returns { "collection": "string", "name": "string", ...}
            # We just pass it to update
            update_opensea_collection_data(db_path, table_name, slug, data)
            
        # optional short sleep to avoid rate-limiting
        # time.sleep(0.2)

    print("Done updating additional data for all OpenSea collections in random_sample_5.db.")

In [ ]:
if __name__ == "__main__":
    OPENSEA_API_KEY = "your_key"
    fetch_and_update_opensea_collection_data(
        db_path="random_sample_5.db",
        table_name="sampled_collections",
        api_key=OPENSEA_API_KEY
    )

Added column 'total_supply' to sampled_collections
[1/14] Fetching extended data for slug='mekaapes-blast'
[2/14] Fetching extended data for slug='cincinnati-physicians'
[3/14] Fetching extended data for slug='dump-pepe-5'
[4/14] Fetching extended data for slug='zora-posts-21385'
[5/14] Fetching extended data for slug='me-physical-collectibles-10'
[6/14] Fetching extended data for slug='aaaaaa-26'
[7/14] Fetching extended data for slug='giants-g-nine-2024-emblem'
[8/14] Fetching extended data for slug='zora-posts-19156'
[9/14] Fetching extended data for slug='bbbdf-e'
[10/14] Fetching extended data for slug='mintak-2'
[11/14] Fetching extended data for slug='lens-collect-profile-126134-publication-890'
[12/14] Fetching extended data for slug='city-s-art'
[13/14] Fetching extended data for slug='blueandpink'
[14/14] Fetching extended data for slug='lens-collect-profile-143257-publication-8280'
Done updating additional data for all OpenSea collections in random_sample_5.db.


In [14]:
def add_traits_json_column_if_not_exists(db_path="random_sample_5.db", table_name="sampled_collections"):
    """
    Adds a 'traits_json' column to the 'sampled_collections' table if it does not exist.
    """
    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    
    # Attempt an ALTER TABLE for 'traits_json'
    alter_stmt = f"ALTER TABLE {table_name} ADD COLUMN traits_json TEXT"
    try:
        c.execute(alter_stmt)
        print(f"Added 'traits_json' column to {table_name}")
    except sqlite3.OperationalError:
        # Column probably already exists
        pass
    
    conn.commit()
    conn.close()

In [15]:
def fetch_opensea_traits(collection_slug, api_key):
    """
    Calls the OpenSea endpoint:
    GET /api/v2/traits/{collection_slug}
    
    Returns JSON like:
      {
        "categories": {...},
        "counts": {...}
      }
    """
    url = f"https://api.opensea.io/api/v2/traits/{collection_slug}"
    headers = {
        "accept": "application/json",
        "x-api-key": api_key
    }
    resp = requests.get(url, headers=headers)
    if resp.status_code == 200:
        return resp.json()
    else:
        print(f"[OpenSea Traits] Error {resp.status_code} for slug='{collection_slug}': {resp.text}")
        return {}


def store_traits_json_in_collections(db_path, table_name, slug_name, traits_data):
    """
    Updates the 'sampled_collections' table with the entire traits JSON
    in the 'traits_json' column for the given slug_name.
    
    We assume marketplace='OpenSea' is the row filter.
    """
    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    
    update_stmt = f"""
    UPDATE {table_name}
    SET traits_json = ?
    WHERE marketplace='OpenSea'
      AND slug_name=?
    """
    c.execute(update_stmt, (json.dumps(traits_data), slug_name))
    conn.commit()
    conn.close()


def fetch_and_update_opensea_traits_in_collections(db_path="random_sample_5.db",
                                                   table_name="sampled_collections",
                                                   api_key="YOUR_API_KEY"):
    """
    1) Ensure 'traits_json' column exists in 'sampled_collections'.
    2) Query all rows where marketplace='OpenSea' and slug_name is not null.
    3) For each slug_name, fetch traits JSON from /api/v2/traits/{slug_name} and
       store it in the 'traits_json' column.
    """
    # 1) Add 'traits_json' column if needed
    add_traits_json_column_if_not_exists(db_path, table_name)
    
    # 2) Query the existing table for OpenSea slugs
    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    c.execute(f"""
    SELECT slug_name
    FROM {table_name}
    WHERE marketplace='OpenSea'
      AND slug_name IS NOT NULL
    """)
    rows = c.fetchall()
    conn.close()
    
    # 3) For each slug, fetch traits and store
    for idx, (slug,) in enumerate(rows, start=1):
        print(f"[{idx}/{len(rows)}] Fetching traits for slug='{slug}'")
        traits_data = fetch_opensea_traits(slug, api_key)
        store_traits_json_in_collections(db_path, table_name, slug, traits_data)
    
    print(f"Done updating 'traits_json' for all OpenSea collections in {table_name}.")

In [ ]:
if __name__ == "__main__":
    OPENSEA_API_KEY = "your_key"
    fetch_and_update_opensea_traits_in_collections(
        db_path="random_sample_5.db",
        table_name="sampled_collections",
        api_key=OPENSEA_API_KEY
    )

Added 'traits_json' column to sampled_collections
[1/14] Fetching traits for slug='mekaapes-blast'
[2/14] Fetching traits for slug='cincinnati-physicians'
[3/14] Fetching traits for slug='dump-pepe-5'
[4/14] Fetching traits for slug='zora-posts-21385'
[5/14] Fetching traits for slug='me-physical-collectibles-10'
[6/14] Fetching traits for slug='aaaaaa-26'
[7/14] Fetching traits for slug='giants-g-nine-2024-emblem'
[8/14] Fetching traits for slug='zora-posts-19156'
[9/14] Fetching traits for slug='bbbdf-e'
[10/14] Fetching traits for slug='mintak-2'
[11/14] Fetching traits for slug='lens-collect-profile-126134-publication-890'
[12/14] Fetching traits for slug='city-s-art'
[13/14] Fetching traits for slug='blueandpink'
[14/14] Fetching traits for slug='lens-collect-profile-143257-publication-8280'
Done updating 'traits_json' for all OpenSea collections in sampled_collections.


Rarible

In [17]:
def transform_collection_id(collection_id_blockchain):
    """
    Given a string like '0x594824a3d6e5777b3c7cc202ad1050435aac7698:ethereum',
    transform it into 'ETHEREUM:0x594824a3d6e5777b3c7cc202ad1050435aac7698'
    for Rarible's endpoint: ?collectionIds=ETHEREUM%3A0x..
    """
    if ":" not in collection_id_blockchain:
        return None
    coll_id, chain = collection_id_blockchain.split(":")
    return f"{chain.upper()}:{coll_id}"

def fetch_rarible_traits_single(collection_id_rarible, api_key):
    """
    Calls Rarible's endpoint, single collection ID:
      GET /v0.1/items/traits?collectionIds={collection_id_rarible}

    Returns the JSON with structure:
      {
        "continuation": "string",
        "traits": [ { "key": {...}, "values": [...] }, ... ]
      }
    """
    base_url = "https://api.rarible.org/v0.1/items/traits"
    headers = {
        "accept": "application/json",
        "X-API-KEY": api_key
    }
    # For a single ID, just pass collectionIds once
    params = {
        "collectionIds": collection_id_rarible
    }
    resp = requests.get(base_url, headers=headers, params=params)
    if resp.status_code == 200:
        return resp.json()
    else:
        print(f"[Rarible Traits] Error {resp.status_code} => {resp.text}")
        return {}

def update_traits_json_for_rarible(db_path="random_sample_5.db",
                                   table_name="sampled_collections",
                                   api_key="YOUR_RARIBLE_API_KEY"):
    """
    1) Query 'sampled_collections' where marketplace='Rarible'
    2) For each row, transform 'collection_id' to Rarible format, fetch traits,
       store entire JSON in 'traits_json' column
    """
    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    
    # We'll assume 'traits_json' column already exists (like for OpenSea).
    # If not, you'll need the add_column logic from your previous script.
    
    c.execute(f"""
    SELECT collection_id
    FROM {table_name}
    WHERE marketplace='Rarible'
      AND collection_id IS NOT NULL
    """)
    rows = c.fetchall()
    conn.close()
    
    print(f"Found {len(rows)} Rarible collections to fetch traits for.")
    
    count = 0
    for (orig_id,) in rows:
        rarible_id = transform_collection_id(orig_id)
        if rarible_id is None:
            print(f"Skipping invalid format => {orig_id}")
            continue
        
        data = fetch_rarible_traits_single(rarible_id, api_key)
        
        # Now update the 'traits_json' in that row
        conn2 = sqlite3.connect(db_path)
        c2 = conn2.cursor()
        update_stmt = f"""
        UPDATE {table_name}
        SET traits_json = ?
        WHERE marketplace='Rarible'
          AND collection_id=?
        """
        c2.execute(update_stmt, (json.dumps(data), orig_id))
        conn2.commit()
        conn2.close()
        
        count += 1
        print(f"[{count}/{len(rows)}] Updated 'traits_json' for => {orig_id}")
    
    print("Done fetching/storing Rarible traits in 'traits_json' column.")

In [ ]:
if __name__ == "__main__":
    RARIBLE_API_KEY = "your_key"
    update_traits_json_for_rarible(
        db_path="random_sample_5.db",
        table_name="sampled_collections",
        api_key=RARIBLE_API_KEY
    )

Found 15 Rarible collections to fetch traits for.
[1/15] Updated 'traits_json' for => 0x716ad1b6222046289c1664825cd9e4caf6253aec:match
[2/15] Updated 'traits_json' for => 0x0c15b61197c8fca4322b8e6c4a744b597622a1dc:base
[3/15] Updated 'traits_json' for => 0x84f2789475754572311d65173ade3c24e643ba29:base
[4/15] Updated 'traits_json' for => 0x97683a99e531201de43f68c445a0325a426ebf7c:celo
[5/15] Updated 'traits_json' for => 0x41ffb8407a23a1c1aa8b948677428e5049b850c1:base
[6/15] Updated 'traits_json' for => 0x482f1759dd48df2672c0b9a5fc6791b19f4fb7d3:etherlink
[7/15] Updated 'traits_json' for => 0x7b25869dd5405b82b0debf48642eaca4e2a61a20:abstract
[8/15] Updated 'traits_json' for => 0xbd88289d94f28cae7a5ea1136be8f78b863bb1cd:base
[9/15] Updated 'traits_json' for => 0x5f5a2e9642d772a6d9b348621d0330ef0bc4222e:zksync
[10/15] Updated 'traits_json' for => 0xeeec1bda08cc5a6a26d43406c08d0d35bd54d830:abstract
[11/15] Updated 'traits_json' for => 0x68de02dab89a266858d74e2ca6707693fff022f16d9d9020f52e08

In [19]:
def create_rarible_nfts_table(db_path="random_sample_5.db", table_name="rarible_nfts"):
    """
    Creates a table 'rarible_nfts' with columns similar to 'opensea_nfts',
    storing key fields from the Rarible item-level data plus a link back 
    to 'collection_id' in 'sampled_collections'.
    """
    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    
    c.execute(f"""
    CREATE TABLE IF NOT EXISTS {table_name} (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        -- references the original 'id:blockchain' format from 'sampled_collections'
        collection_id TEXT,
        
        item_id TEXT,          -- item["id"] e.g. "ETHEREUM:0xb66a6...:123"
        blockchain TEXT,       -- item["blockchain"]
        contract TEXT,         -- item["contract"]
        token_id TEXT,         -- item["tokenId"]
        
        name TEXT,             -- from item["meta"]["name"]
        description TEXT,      -- from item["meta"]["description"]
        image_url TEXT,        -- optional, if we parse item["meta"]["content"] for an image
        minted_at TEXT,        -- item["mintedAt"]
        last_updated TEXT,     -- item["lastUpdatedAt"]
        supply REAL,           -- item["supply"] 
        owner_if_single TEXT,  -- item["ownerIfSingle"] if we want
        
        project_url TEXT,      -- e.g. item["meta"]["externalUri"] if present
        created_at TEXT,       -- item["meta"]["createdAt"]
        
        fetched_at TEXT
    )
    """)
    
    conn.commit()
    conn.close()

In [20]:
def transform_collection_id_for_rarible(orig_id):
    """
    Convert 'id:blockchain' => 'BLOCKCHAIN:0xid' for use in Rarible API.
    e.g. '0x5948abcd:ethereum' => 'ETHEREUM:0x5948abcd'
    """
    if ":" not in orig_id:
        return None
    coll_id, chain = orig_id.split(":")
    return f"{chain.upper()}:{coll_id}"

def fetch_rarible_nfts(collection_param, api_key, size=10):
    """
    Calls Rarible endpoint:
      GET /v0.1/items/byCollection?collection={collection_param}&size={size}
    e.g. collection_param = 'ETHEREUM:0xb66a603f...'
    Returns a dict with keys 'continuation' and 'items' (list).
    """
    url = "https://api.rarible.org/v0.1/items/byCollection"
    headers = {
        "accept": "application/json",
        "X-API-KEY": api_key
    }
    params = {
        "collection": collection_param,
        "size": size
    }
    resp = requests.get(url, headers=headers, params=params)
    if resp.status_code == 200:
        return resp.json()
    else:
        print(f"[Rarible NFT Fetch] Error {resp.status_code} => {resp.text}")
        return {}

def store_rarible_nfts(db_path, table_name, orig_id, items_list):
    """
    Insert up to 10 items into 'rarible_nfts' table, storing relevant fields.
    :param orig_id: The original 'id:blockchain' used in 'sampled_collections'
    :param items_list: list of item dicts from the Rarible API
    """
    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    
    insert_sql = f"""
    INSERT INTO {table_name} (
        collection_id,
        item_id,
        blockchain,
        contract,
        token_id,
        name,
        description,
        image_url,
        minted_at,
        last_updated,
        supply,
        owner_if_single,
        project_url,
        created_at,
        fetched_at
    )
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, datetime('now'))
    """
    
    for item in items_list:
        item_id = item.get("id", "")
        blockchain = item.get("blockchain", "")
        contract = item.get("contract", "")
        token_id = str(item.get("tokenId", ""))
        supply = item.get("supply", 0)
        minted_at = item.get("mintedAt", "")
        last_updated = item.get("lastUpdatedAt", "")
        owner_if_single = item.get("ownerIfSingle", "")
        
        meta = item.get("meta", {})
        name = meta.get("name", "")
        description = meta.get("description", "")
        created_at = meta.get("createdAt", "")
        project_url = meta.get("externalUri", "")
        
        # Optional: parse meta["content"] for image
        # The example doesn't show a direct "url", so we may store 'N/A' or parse further
        content_list = meta.get("content", [])
        image_url = "N/A"
        for content_obj in content_list:
            # if there's a recognized URL, parse it
            # The example lacks a 'url' field, so we skip. 
            # If your real data has it, you might do:
            # image_url = content_obj.get("url", "N/A")
            # break
            pass
        
        c.execute(insert_sql, (
            orig_id,      # collection_id (as it appears in 'sampled_collections')
            item_id,
            blockchain,
            contract,
            token_id,
            name,
            description,
            image_url,
            minted_at,
            last_updated,
            supply,
            owner_if_single,
            project_url,
            created_at
        ))
    
    conn.commit()
    conn.close()

def fetch_rarible_nfts_for_collections(db_path="random_sample_5.db", table_collections="sampled_collections",
                                       table_nfts="rarible_nfts", api_key="YOUR_RARIBLE_API_KEY", size=10):
    """
    1) Create the 'rarible_nfts' table if not exists.
    2) Query 'sampled_collections' where marketplace='Rarible'.
    3) Transform 'collection_id:blockchain' to 'BLOCKCHAIN:collection_id'.
    4) Fetch items (size=10) from Rarible's endpoint.
    5) Store them in 'rarible_nfts'.
    """
    # 1) create table
    create_rarible_nfts_table(db_path, table_nfts)
    
    # 2) query the relevant rows
    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    c.execute(f"""
    SELECT collection_id
    FROM {table_collections}
    WHERE marketplace='Rarible'
      AND collection_id IS NOT NULL
    """)
    rows = c.fetchall()
    conn.close()
    
    count = 0
    for (orig_id,) in rows:
        rarible_format = transform_collection_id_for_rarible(orig_id)
        if not rarible_format:
            print(f"Skipping invalid format => {orig_id}")
            continue
        
        data = fetch_rarible_nfts(rarible_format, api_key, size=size)
        items = data.get("items", [])
        
        store_rarible_nfts(db_path, table_nfts, orig_id, items)
        count += 1
        print(f"[{count}/{len(rows)}] Inserted up to {len(items)} items for {orig_id}")
    
    print("Done fetching and storing Rarible NFTs in", table_nfts)

In [ ]:
if __name__ == "__main__":
    RARIBLE_API_KEY = "your_key"
    fetch_rarible_nfts_for_collections(
        db_path="random_sample_5.db",
        table_collections="sampled_collections",
        table_nfts="rarible_nfts",
        api_key=RARIBLE_API_KEY,
        size=10
    )

[1/15] Inserted up to 10 items for 0x716ad1b6222046289c1664825cd9e4caf6253aec:match
[2/15] Inserted up to 10 items for 0x0c15b61197c8fca4322b8e6c4a744b597622a1dc:base
[3/15] Inserted up to 1 items for 0x84f2789475754572311d65173ade3c24e643ba29:base
[4/15] Inserted up to 1 items for 0x97683a99e531201de43f68c445a0325a426ebf7c:celo
[5/15] Inserted up to 1 items for 0x41ffb8407a23a1c1aa8b948677428e5049b850c1:base
[6/15] Inserted up to 1 items for 0x482f1759dd48df2672c0b9a5fc6791b19f4fb7d3:etherlink
[7/15] Inserted up to 0 items for 0x7b25869dd5405b82b0debf48642eaca4e2a61a20:abstract
[8/15] Inserted up to 1 items for 0xbd88289d94f28cae7a5ea1136be8f78b863bb1cd:base
[9/15] Inserted up to 1 items for 0x5f5a2e9642d772a6d9b348621d0330ef0bc4222e:zksync
[10/15] Inserted up to 0 items for 0xeeec1bda08cc5a6a26d43406c08d0d35bd54d830:abstract
[11/15] Inserted up to 1 items for 0x68de02dab89a266858d74e2ca6707693fff022f16d9d9020f52e08271acade84:aptos
[12/15] Inserted up to 1 items for 0x9fcd53e9d106ceb6

MagicEden

In [22]:
def fetch_magiceden_attributes(slug_name):
    base_url = "https://api-mainnet.magiceden.dev/v2/collections"
    url = f"{base_url}/{slug_name}/attributes"
    headers = {"accept": "application/json"}
    
    resp = requests.get(url, headers=headers)
    if resp.status_code == 200:
        return resp.json()
    else:
        print(f"[MagicEden Attributes] Error {resp.status_code} for slug='{slug_name}': {resp.text}")
        return {}

def store_traits_json_for_magiceden(db_path, table_name, slug_name, traits_data):
    """
    Updates the 'sampled_collections' row for marketplace='MagicEden'
    and the given slug_name, setting the entire JSON in 'traits_json'.
    """
    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    
    update_sql = f"""
    UPDATE {table_name}
    SET traits_json = ?
    WHERE marketplace='MagicEden'
      AND slug_name=?
    """
    c.execute(update_sql, (json.dumps(traits_data), slug_name))
    
    conn.commit()
    conn.close()

def fetch_and_update_magiceden_traits(db_path="random_sample_5.db", table_name="sampled_collections"):
    """
    1) Query 'sampled_collections' for rows with marketplace='MagicEden'.
    2) For each 'slug_name', call fetch_magiceden_attributes(slug_name).
    3) Store the JSON result in 'traits_json' column of the same row.
    """
    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    
    # We'll fetch all MagicEden slug_names
    c.execute(f"""
    SELECT slug_name
    FROM {table_name}
    WHERE marketplace='MagicEden'
      AND slug_name IS NOT NULL
    """)
    rows = c.fetchall()
    conn.close()
    
    print(f"Found {len(rows)} Magic Eden collections to update 'traits_json'.")
    
    count = 0
    for (slug_name,) in rows:
        data = fetch_magiceden_attributes(slug_name)
        store_traits_json_for_magiceden(db_path, table_name, slug_name, data)
        count += 1
        print(f"[{count}/{len(rows)}] Updated 'traits_json' for slug='{slug_name}'")
    
    print("Done fetching/storing Magic Eden attributes in 'traits_json'.")

In [23]:
if __name__ == "__main__":
    fetch_and_update_magiceden_traits(
        db_path="random_sample_5.db",
        table_name="sampled_collections"
    )

Found 8 Magic Eden collections to update 'traits_json'.
[1/8] Updated 'traits_json' for slug='spotted_lanternflies'
[2/8] Updated 'traits_json' for slug='kyoudai_spirits'
[3/8] Updated 'traits_json' for slug='solana_mecha_apes'
[4/8] Updated 'traits_json' for slug='drip_genopets_s2'
[5/8] Updated 'traits_json' for slug='pixelated_apes_dao'
[6/8] Updated 'traits_json' for slug='De_Casinos'
[7/8] Updated 'traits_json' for slug='moveman'
[8/8] Updated 'traits_json' for slug='frostys_friends'
Done fetching/storing Magic Eden attributes in 'traits_json'.


In [24]:
def create_magiceden_nfts_table(db_path="random_sample_5.db", table_name="magiceden_nfts"):
    """
    Creates a table 'magiceden_nfts' with columns for relevant fields from
    the Magic Eden listing JSON, plus a link back to 'sampled_collections' via slug_name.
    
    We'll have 27 columns total:
      1) id (PK)
      2) collection_slug
      ...
      26) token_properties_json
      27) fetched_at
    """
    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    
    # Below we define 27 columns total
    c.execute(f"""
    CREATE TABLE IF NOT EXISTS {table_name} (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        
        collection_slug TEXT,

        pdaAddress TEXT,
        auctionHouse TEXT,
        tokenAddress TEXT,
        tokenMint TEXT,
        seller TEXT,
        sellerReferral TEXT,
        tokenSize REAL,
        price REAL,
        expiry REAL,
        
        rarity_json TEXT,
        extra_json TEXT,
        listingSource TEXT,
        
        token_mintAddress TEXT,
        token_owner TEXT,
        token_supply REAL,
        token_collection TEXT,
        token_name TEXT,
        token_updateAuthority TEXT,
        token_primarySaleHappened INTEGER,
        token_sellerFeeBasisPoints REAL,
        token_image TEXT,
        token_animationUrl TEXT,
        token_externalUrl TEXT,
        
        token_attributes_json TEXT,
        token_properties_json TEXT,
        
        fetched_at TEXT
    )
    """)
    
    conn.commit()
    conn.close()

def fetch_magiceden_listings(collection_slug, limit=10):
    """
    GET https://api-mainnet.magiceden.dev/v2/collections/{collection_slug}/listings?limit={limit}
    Returns a list of dicts if successful, else empty list.
    """
    base_url = "https://api-mainnet.magiceden.dev/v2/collections"
    url = f"{base_url}/{collection_slug}/listings"
    params = {"limit": limit}
    headers = {"accept": "application/json"}
    
    resp = requests.get(url, headers=headers, params=params)
    if resp.status_code == 200:
        return resp.json()  # Should be a list of listing dicts
    else:
        print(f"[MagicEden NFT Fetch] Error {resp.status_code} for slug='{collection_slug}': {resp.text}")
        return []

def store_magiceden_nfts(db_path, table_name, slug_name, listings):
    """
    Insert each listing item into 'magiceden_nfts'.
    We must provide EXACTLY 26 placeholders for these columns (excluding 'id' + including 'fetched_at' as a datetime call).
    """
    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    
    # 27 columns total => 'id' is autoincrement, so we have 26 placeholders
    insert_sql = f"""
    INSERT INTO {table_name} (
        collection_slug,
        
        pdaAddress,
        auctionHouse,
        tokenAddress,
        tokenMint,
        seller,
        sellerReferral,
        tokenSize,
        price,
        expiry,
        
        rarity_json,
        extra_json,
        listingSource,
        
        token_mintAddress,
        token_owner,
        token_supply,
        token_collection,
        token_name,
        token_updateAuthority,
        token_primarySaleHappened,
        token_sellerFeeBasisPoints,
        token_image,
        token_animationUrl,
        token_externalUrl,
        
        token_attributes_json,
        token_properties_json,
        
        fetched_at
    )
    VALUES (
        ?,  -- collection_slug
        ?,  -- pdaAddress
        ?,  -- auctionHouse
        ?,  -- tokenAddress
        ?,  -- tokenMint
        ?,  -- seller
        ?,  -- sellerReferral
        ?,  -- tokenSize
        ?,  -- price
        ?,  -- expiry
        ?,  -- rarity_json
        ?,  -- extra_json
        ?,  -- listingSource
        ?,  -- token_mintAddress
        ?,  -- token_owner
        ?,  -- token_supply
        ?,  -- token_collection
        ?,  -- token_name
        ?,  -- token_updateAuthority
        ?,  -- token_primarySaleHappened
        ?,  -- token_sellerFeeBasisPoints
        ?,  -- token_image
        ?,  -- token_animationUrl
        ?,  -- token_externalUrl
        ?,  -- token_attributes_json
        ?,  -- token_properties_json
        datetime('now')  -- fetched_at
    )
    """
    # Notice we have exactly 26 placeholders (the last column uses datetime('now')).

    for listing in listings:
        pdaAddress = listing.get("pdaAddress", "")
        auctionHouse = listing.get("auctionHouse", "")
        tokenAddress = listing.get("tokenAddress", "")
        tokenMint = listing.get("tokenMint", "")
        seller = listing.get("seller", "")
        sellerReferral = listing.get("sellerReferral", "")
        tokenSize = listing.get("tokenSize", 0)
        price = listing.get("price", 0)
        expiry = listing.get("expiry", 0)
        
        # Convert 'rarity' and 'extra' to JSON strings
        rarity_json = json.dumps(listing.get("rarity", {}))
        extra_json = json.dumps(listing.get("extra", {}))
        
        listingSource = listing.get("listingSource", "")
        
        token_obj = listing.get("token", {})
        token_mintAddress = token_obj.get("mintAddress", "")
        token_owner = token_obj.get("owner", "")
        token_supply = token_obj.get("supply", 0)
        token_collection = token_obj.get("collection", "")
        token_name = token_obj.get("name", "")
        token_updateAuthority = token_obj.get("updateAuthority", "")
        
        primarySaleHappened = 1 if token_obj.get("primarySaleHappened", False) else 0
        sellerFeeBasisPoints = token_obj.get("sellerFeeBasisPoints", 0)
        token_image = token_obj.get("image", "")
        token_animationUrl = token_obj.get("animationUrl", "")
        token_externalUrl = token_obj.get("externalUrl", "")
        
        attributes_json = json.dumps(token_obj.get("attributes", []))
        properties_json = json.dumps(token_obj.get("properties", {}))
        
        c.execute(insert_sql, (
            slug_name,
            pdaAddress,
            auctionHouse,
            tokenAddress,
            tokenMint,
            seller,
            sellerReferral,
            tokenSize,
            price,
            expiry,
            rarity_json,
            extra_json,
            listingSource,
            token_mintAddress,
            token_owner,
            token_supply,
            token_collection,
            token_name,
            token_updateAuthority,
            primarySaleHappened,
            sellerFeeBasisPoints,
            token_image,
            token_animationUrl,
            token_externalUrl,
            attributes_json,
            properties_json
            # fetched_at => datetime('now') in SQL
        ))
    
    conn.commit()
    conn.close()

def fetch_magiceden_nfts_for_collections(db_path="random_sample_5.db",
                                         table_collections="sampled_collections",
                                         table_nfts="magiceden_nfts",
                                         limit=10):
    """
    Creates table if necessary, then for each row with marketplace='MagicEden',
    uses 'slug_name' to fetch up to {limit} items from Magic Eden,
    storing them in magiceden_nfts.
    """
    create_magiceden_nfts_table(db_path, table_nfts)
    
    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    c.execute(f"""
    SELECT slug_name
    FROM {table_collections}
    WHERE marketplace='MagicEden'
      AND slug_name IS NOT NULL
    """)
    rows = c.fetchall()
    conn.close()
    
    count = 0
    for (slug_name,) in rows:
        data = fetch_magiceden_listings(slug_name, limit=limit)
        store_magiceden_nfts(db_path, table_nfts, slug_name, data)
        count += 1
        print(f"[{count}/{len(rows)}] Inserted up to {len(data)} items for MagicEden => {slug_name}")
    
    print(f"Done fetching/storing MagicEden listings in {table_nfts}")

In [25]:
if __name__ == "__main__":
    fetch_magiceden_nfts_for_collections(
        db_path="random_sample_5.db",
        table_collections="sampled_collections",
        table_nfts="magiceden_nfts",
        limit=10
    )

[1/8] Inserted up to 7 items for MagicEden => spotted_lanternflies
[2/8] Inserted up to 10 items for MagicEden => kyoudai_spirits
[3/8] Inserted up to 10 items for MagicEden => solana_mecha_apes
[4/8] Inserted up to 10 items for MagicEden => drip_genopets_s2
[5/8] Inserted up to 0 items for MagicEden => pixelated_apes_dao
[6/8] Inserted up to 10 items for MagicEden => De_Casinos
[7/8] Inserted up to 10 items for MagicEden => moveman
[8/8] Inserted up to 10 items for MagicEden => frostys_friends
Done fetching/storing MagicEden listings in magiceden_nfts


Atomic

In [26]:
def fetch_atomic_collection_stats(collection_name):
    """
    Calls /atomicassets/v1/collections/{collection_name}/stats"
    """
    url = f"https://wax.api.atomicassets.io/atomicassets/v1/collections/{collection_name}/stats"
    resp = requests.get(url)
    if resp.status_code == 200:
        return resp.json()
    else:
        print(f"[Atomic Stats] Error {resp.status_code} for collection='{collection_name}': {resp.text}")
        return {}

def update_atomic_collection_total_supply(db_path="random_sample_5.db",
                                          table_name="sampled_collections"):
    """
    1) Query 'sampled_collections' where marketplace='Atomic',
       using 'slug_name' as the collection_name.
    2) For each row, call fetch_atomic_collection_stats(...),
       parse data["data"]["assets"], and store it in 'total_supply'.
    """
    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    
    c.execute(f"""
    SELECT slug_name
    FROM {table_name}
    WHERE marketplace='Atomic'
      AND slug_name IS NOT NULL
    """)
    rows = c.fetchall()
    conn.close()
    
    print(f"Found {len(rows)} 'Atomic' collections to update total_supply.")
    
    count = 0
    for (collection_name,) in rows:
        stats_data = fetch_atomic_collection_stats(collection_name)
        # stats_data => { "success": bool, "data": {...}, ... }
        if not stats_data.get("success", False):
            print(f"Skipping {collection_name}, 'success' is False or missing.")
            continue
        
        # If "assets" is None or missing, default to "0"
        assets_str = stats_data.get("data", {}).get("assets")
        if assets_str is None:
            assets_str = "0"
        
        try:
            assets_count = int(assets_str)
        except ValueError:
            assets_count = 0  # or handle differently if needed
        
        conn2 = sqlite3.connect(db_path)
        c2 = conn2.cursor()
        update_stmt = f"""
        UPDATE {table_name}
        SET total_supply = ?
        WHERE marketplace='Atomic'
          AND slug_name=?
        """
        c2.execute(update_stmt, (assets_count, collection_name))
        conn2.commit()
        conn2.close()
        
        count += 1
        print(f"[{count}/{len(rows)}] Updated total_supply={assets_count} for {collection_name}")
    
    print("Done updating total_supply for Atomic collections.")

In [27]:
if __name__ == "__main__":
    update_atomic_collection_total_supply(
        db_path="random_sample_5.db",
        table_name="sampled_collections"
    )

Found 3 'Atomic' collections to update total_supply.
[1/3] Updated total_supply=10 for nftartdesing
[2/3] Updated total_supply=20 for targetedby11
[3/3] Updated total_supply=5 for alienwor1dsz
Done updating total_supply for Atomic collections.


In [28]:
def create_atomic_nfts_table(db_path="random_sample_5.db", table_name="atomic_nfts"):
    """
    Creates a table that stores item-level data from the AtomicAssets API.
    Adjust columns as needed to match your chosen fields.
    """
    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    
    c.execute(f"""
    CREATE TABLE IF NOT EXISTS {table_name} (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        
        -- references the original 'collection_name' from 'sampled_collections' (Atomic)
        collection_name TEXT,
        
        asset_id TEXT,
        owner TEXT,
        item_name TEXT,
        is_transferable INTEGER,
        is_burnable INTEGER,
        
        template_id TEXT,        -- from template["template_id"]
        max_supply TEXT,         -- from template["max_supply"]
        issued_supply TEXT,      -- from template["issued_supply"]
        
        minted_at_time TEXT,     -- item["minted_at_time"]
        minted_at_block TEXT,    -- item["minted_at_block"]
        
        data_json TEXT,          -- optionally store the entire item if you like
        fetched_at TEXT
    )
    """)
    
    conn.commit()
    conn.close()

In [29]:
def fetch_atomic_assets(collection_name, limit=10):
    """
    Calls the endpoint:
      GET /atomicassets/v1/assets?collection_name={collection_name}&limit={limit}
    Returns a dict with "success", "data" (list), and "query_time".
    We'll return the entire JSON for reference.
    """
    base_url = "https://wax.api.atomicassets.io/atomicassets/v1/assets"
    params = {
        "collection_name": collection_name,
        "limit": limit
    }
    resp = requests.get(base_url, params=params)
    if resp.status_code == 200:
        return resp.json()  # e.g. { "success": true, "data": [...], "query_time": 0 }
    else:
        print(f"[Atomic NFT Fetch] Error {resp.status_code} => {resp.text}")
        return {}

def store_atomic_nfts(db_path, table_name, coll_name, items):
    """
    Inserts the provided item-level data into 'atomic_nfts'.
    items is a list of dicts from data["data"] in the AtomicAssets response.
    """
    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    
    insert_sql = f"""
    INSERT INTO {table_name} (
        collection_name,
        asset_id,
        owner,
        item_name,
        is_transferable,
        is_burnable,
        template_id,
        max_supply,
        issued_supply,
        minted_at_time,
        minted_at_block,
        data_json,
        fetched_at
    )
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, datetime('now'))
    """
    
    for item in items:
        asset_id = item.get("asset_id", "")
        owner = item.get("owner", "")
        item_name = item.get("name", "")
        
        # Convert booleans to int
        is_transferable = 1 if item.get("is_transferable", False) else 0
        is_burnable = 1 if item.get("is_burnable", False) else 0
        
        # Safely handle template, which can be None
        template = item.get("template") or {}
        template_id = template.get("template_id", "")
        max_supply = template.get("max_supply", "")
        issued_supply = template.get("issued_supply", "")
        
        minted_at_time = item.get("minted_at_time", "")
        minted_at_block = item.get("minted_at_block", "")
        
        data_json = json.dumps(item)
        
        c.execute(insert_sql, (
            coll_name,
            asset_id,
            owner,
            item_name,
            is_transferable,
            is_burnable,
            template_id,
            max_supply,
            issued_supply,
            minted_at_time,
            minted_at_block,
            data_json
        ))
    
    conn.commit()
    conn.close()
    
def fetch_atomic_nfts_for_collections(db_path="random_sample_5.db",
                                      table_collections="sampled_collections",
                                      table_nfts="atomic_nfts",
                                      limit=10):
    """
    1) Create the 'atomic_nfts' table if not exists.
    2) Query 'sampled_collections' for marketplace='Atomic'.
       We'll assume there's a column 'collection_name' for each row.
    3) For each row, call fetch_atomic_assets(collection_name, limit={limit}),
       then store them in 'atomic_nfts'.
    """
    # 1) create table
    create_atomic_nfts_table(db_path, table_nfts)
    
    # 2) get the relevant rows from 'sampled_collections'
    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    c.execute(f"""
    SELECT slug_name
    FROM {table_collections}
    WHERE marketplace='Atomic'
      AND slug_name IS NOT NULL
    """)
    rows = c.fetchall()
    conn.close()
    
    print(f"Found {len(rows)} Atomic collections to fetch up to {limit} assets each.")
    
    count = 0
    for (coll_name,) in rows:
        print(f"[{count+1}/{len(rows)}] Fetching {limit} assets for collection_name='{coll_name}'")
        
        data = fetch_atomic_assets(coll_name, limit=limit)
        items = data.get("data", [])  # a list of item dicts
        
        store_atomic_nfts(db_path, table_nfts, coll_name, items)
        count += 1
    
    print("Done fetching/storing Atomic NFTs in", table_nfts)

In [30]:
if __name__ == "__main__":
    fetch_atomic_nfts_for_collections(
        db_path="random_sample_5.db",
        table_collections="sampled_collections",
        table_nfts="atomic_nfts",
        limit=10
    )

Found 3 Atomic collections to fetch up to 10 assets each.
[1/3] Fetching 10 assets for collection_name='nftartdesing'
[2/3] Fetching 10 assets for collection_name='targetedby11'
[3/3] Fetching 10 assets for collection_name='alienwor1dsz'
Done fetching/storing Atomic NFTs in atomic_nfts
